In [9]:
import sqlite3
import pandas as pd

# Conectando ao banco de dados SQLite
caminho_db = "../database.db"
conexao = sqlite3.connect(caminho_db)

# Cria um dataframe para cada tabela
cpu = pd.read_sql_query("SELECT * FROM cpu", conexao)
memory = pd.read_sql_query("SELECT * FROM memory", conexao)
hd = pd.read_sql_query("SELECT * FROM hd", conexao)
ssd = pd.read_sql_query("SELECT * FROM ssd", conexao)
psu = pd.read_sql_query("SELECT * FROM psu", conexao)
computerCase = pd.read_sql_query("SELECT * FROM computerCase", conexao)
cooler = pd.read_sql_query("SELECT * FROM cooler", conexao)
gpu = pd.read_sql_query("SELECT * FROM gpu", conexao)
motherboard = pd.read_sql_query("SELECT * FROM motherboard", conexao)

# Fechando a conexão
conexao.close()


In [7]:
cpu = cpu.drop_duplicates()
memory = memory.drop_duplicates()
hd = hd.drop_duplicates()
ssd = ssd.drop_duplicates()
psu = psu.drop_duplicates()
computerCase = computerCase.drop_duplicates()
cooler = cooler.drop_duplicates()
gpu = gpu.drop_duplicates()
motherboard = motherboard.drop_duplicates()

In [27]:
import psycopg2

# Parâmetros de conexão
usuario = 'postgres'
senha = '843324'
host = 'localhost'
porta = '5432'
novo_banco = 'pc_assembly'
caminho_sql = '../modelo_pc_assembly.sql'

# 1. Cria o banco de dados (como já faz)
con = psycopg2.connect(
    dbname='postgres',
    user=usuario,
    password=senha,
    host=host,
    port=porta
)
con.autocommit = True
cur = con.cursor()
try:
    cur.execute(f'CREATE DATABASE {novo_banco};')
except psycopg2.Error:
    print("Banco de dados já existe")
cur.close()
con.close()

# 2. Executa o script SQL no novo banco usando psycopg2 puro
con = psycopg2.connect(
    dbname=novo_banco,
    user=usuario,
    password=senha,
    host=host,
    port=porta
)
cur = con.cursor()
with open(caminho_sql, 'r', encoding='utf-8') as f:
    sql_script = f.read()
cur.execute(sql_script)  # Executa todo o script de uma vez
con.commit()
cur.close()
con.close()

print('Banco de dados e tabelas criados com sucesso!')

Banco de dados já existe
Banco de dados e tabelas criados com sucesso!


In [17]:
# Verifica se as tabelas foram criadas no banco 'pc_assembly'
from sqlalchemy import inspect

insp = inspect(engine)
tabelas_criadas = insp.get_table_names()
print("Tabelas criadas no banco 'pc_assembly':")
for tabela in tabelas_criadas:
    print(tabela)


Tabelas criadas no banco 'pc_assembly':


In [18]:
from sqlalchemy import create_engine, text

# Substitua pelos seus dados de acesso
usuario = 'postgres'
senha = '843324'
host = 'localhost'
porta = '5432'
banco = 'pc_assembly'

# Cria a engine de conexão
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}')

# Testando a conexão
with engine.connect() as conexao:
    resultado = conexao.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"))
    for row in resultado:
        print(row)